In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_parquet("hf://datasets/shahules786/PoetryFoundationData/data/train-00000-of-00001-486832872ed96d17.parquet")
print(f"\n\n======\n\n")

print(df.columns)

newyork = df[df['author'].isin(["John Ashbery", "Barbara Guest", "James Schuyler", "Kenneth Koch", "Frank O'Hara"])]
shake = df[df['author'] == 'William Shakespeare']

print(f"Shakespeare: {len(shake)} examples\nNew Yorkers: {len(newyork)} examples")
print(f"Shakespeare avg length: {np.average([len(poem) for poem in shake['content']])}\nNew Yorkers avg length: {np.average([len(poem) for poem in newyork['content']])}")

C:\Users\Anthony\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm






Index(['poem name', 'content', 'author', 'type', 'age'], dtype='object')
Shakespeare: 85 examples
New Yorkers: 81 examples
Shakespeare avg length: 1468.5058823529412
New Yorkers avg length: 1810.6049382716049


In [4]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each length 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-50")
    # wv_from_bin = api.load("glove.6B/glove.6B.50d.txt")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin
wv_from_bin = load_embedding_model()

Loaded vocab size 400000


In [5]:
def process_poem_debug(poem) :
  out = re.sub(r'[\r\n]+', ' ', poem)
  out = re.sub(r'[.?!]+', '.', out)
  out = re.sub(r'\s+', ' ', out)
  out = out.lower()
  sentence_list = out.split('.')
  sentence_list = [sentence for sentence in sentence_list if len(sentence) > 0]
  print(sentence_list)
  for i in range(len(sentence_list)) :
     sentence_list[i] = re.sub(r'[^a-zA-Z ]', '', sentence_list[i])
  return [[word for word in sentence.split(' ') if word != ''] for sentence in sentence_list ]
process_poem_debug("this is a sentence. This is -another :SENTENCE!!!!!\nAND this is a question? again.")

['this is a sentence', ' this is -another :sentence', ' and this is a question', ' again']


[['this', 'is', 'a', 'sentence'],
 ['this', 'is', 'another', 'sentence'],
 ['and', 'this', 'is', 'a', 'question'],
 ['again']]

In [6]:
def process_poem_into_list_of_words(poem) :
  out = re.sub(r'[\r\n]+', ' ', poem)
  out = re.sub(r'[.?!]+', '.', out)
  out = re.sub(r'\s+', ' ', out)
  out = out.lower()
  sentence_list = out.split('.')
  sentence_list = [sentence for sentence in sentence_list if len(sentence) > 0]
  for i in range(len(sentence_list)) :
     sentence_list[i] = re.sub(r'[^a-zA-Z ]', '', sentence_list[i])
  return [[word for word in sentence.split(' ') if word != ''] for sentence in sentence_list ]

newyork_processed = [] # [process_poem_into_list_of_words(newyork['content'].iloc[i]) for i in range(len(newyork))]
for i in range(len(newyork)) :
   newyork_processed += process_poem_into_list_of_words(newyork['content'].iloc[i])
newyork_labels = [0 for i in range(len(newyork_processed))]
shake_processed = [] # [process_poem_into_list_of_words(shake['content'].iloc[i]) for i in range(len(shake))]
for i in range(len(shake)) :
   shake_processed += process_poem_into_list_of_words(shake['content'].iloc[i])
shake_labels = [1 for i in range(len(shake_processed))]

print(f"Number of New Yorker sentences: {len(newyork_processed)} with avg length of {np.mean([len(sentence) for sentence in newyork_processed])}")
print(f"eg:")
for i in range(10) :
   print(f"   {' '.join(newyork_processed[i])}")
print(f"\nNumber of Shakespearean sentences: {len(shake_processed)} with avg length of {np.mean([len(sentence) for sentence in shake_processed])}")
print(f"eg:")
for i in range(10) :
   print(f"   {' '.join(shake_processed[i])}")

processed_poems = newyork_processed + shake_processed
labels = newyork_labels + shake_labels
# perm = np.random.permutation(len(processed_poems))
# processed_poems = processed_poems[perm]
# labels = labels[perm]
newyork_vocab = set([word for poem in newyork_processed for word in poem])
shake_vocab = set([word for poem in shake_processed for word in poem])
vocab = sorted(list(set([word for poem in processed_poems for word in poem])))

print(f"\n\nTotal vocab: {len(vocab)}\nNew York vocab: {len(newyork_vocab)}\nShakespeare Vocab: {len(shake_vocab)}\nIntersection: {len(shake_vocab & newyork_vocab)}")
print(f"New York vocab:")
for i in range(10) :
   print(f"   {list(newyork_vocab)[i]}")
print(f"Shakespeare vocab:")
for i in range(10) :
   print(f"   {list(shake_vocab)[i]}")
print(f"Both vocab:")
for i in range(10) :
   print(f"   {list(newyork_vocab & shake_vocab)[i]}")

word_to_idx = {word:idx for idx, word in enumerate(vocab)}
idx_to_word = {idx:word for idx, word in enumerate(vocab)}

for poem in processed_poems :
  for i in range(len(poem)) :
    poem[i] = word_to_idx[poem[i]]

# print(processed_poems[0])

embedding_matrix = np.zeros((len(vocab), 50))
bad_count = 0
for i, word in enumerate(vocab):
    try:
        embedding_matrix[i] = wv_from_bin.get_vector(word)
    except:
      #   print("this is bad", word)
        bad_count += 1
print(f"\nTotal Bad Words (not in GloVe): {bad_count} out of total vocab {len(vocab)}")

Number of New Yorker sentences: 1412 with avg length of 17.378895184135978
eg:
   is anything central
   orchards flung out on the land urban forests rustic plantations kneehigh hills
   are place names central
   elm grove adcock corner story book farm
   as they concur with a rush at eye level beating themselves into eyes which have had enough thank you no more thank you
   and they come on like scenery mingled with darkness the damp plains overgrown suburbs places of known civic pride of civil obscurity
   these are connected to my version of america but the juice is elsewhere
   this morning as i walked out of your room after breakfast crosshatched with backward and forward glances backward into light forward into unfamiliar light was it our doing and was it the material the lumber of life or of lives we were measuring counting
   a mood soon to be forgotten in crossed girders of light cool downtown shadow in this morning that has seized us again
   i know that i braid too much on 

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentence_lengths = [len(poem) for poem in processed_poems]
max_length = max(sentence_lengths)
avg_length = np.mean(sentence_lengths)
print(f"Max Length = {max_length}\nAvg Length = {avg_length}")
max_length = 50
padded_poems = pad_sequences(processed_poems, maxlen=max_length, padding='post', truncating='post')
print(f"Padded Poems shape is {np.array(padded_poems).shape}")

Max Length = 381
Avg Length = 20.304543409806566
Padded Poems shape is (2223, 50)


In [8]:
perm = np.random.permutation(len(padded_poems))
shuffled_poems = np.array(padded_poems)[perm]
shuffled_labels = np.array(labels)[perm]

training_data = shuffled_poems[:-100]
training_labels = shuffled_labels[:-100]

validation_data = shuffled_poems[-100:]
validation_labels = shuffled_labels[-100:]

## LSTM Discriminator Model

In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

## hyperparams
init_lr = 0.1
lr_decay_rate = 0.5
lr_decay_steps = 100
dropout_p = 0.2
l2_lambda = 0.005

model_lstm = tf.keras.Sequential()
e = Embedding(len(vocab), 50, weights=[embedding_matrix], input_length = max_length, trainable=False)
model_lstm.add(e)
model_lstm.add(LSTM(100, input_shape = (max_length, 50)))
model_lstm.add(Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
model_lstm.add(Dense(50, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
model_lstm.add(Dense(1, activation='sigmoid'))
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(init_lr, decay_rate=lr_decay_rate, decay_steps=lr_decay_steps)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model_lstm.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print(model_lstm.summary())

C:\Users\Anthony\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
C:\Users\Anthony\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)        │ ?                      │       425,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 425,250 (1.62 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 425,250 (1.62 MB)

None


In [79]:
model_lstm.fit(np.array(training_data), np.array(training_labels), epochs=20, verbose=1)

Epoch 1/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 11s 104ms/step - accuracy: 0.5934 - loss: 2.8671
Epoch 2/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 7s 104ms/step - accuracy: 0.6170 - loss: 0.7887
Epoch 3/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 8s 118ms/step - accuracy: 0.6372 - loss: 0.6772
Epoch 4/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 8s 115ms/step - accuracy: 0.6453 - loss: 0.6572
Epoch 5/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 9s 134ms/step - accuracy: 0.6223 - loss: 0.6658
Epoch 6/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 10s 143ms/step - accuracy: 0.6322 - loss: 0.6595
Epoch 7/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 10s 154ms/step - accuracy: 0.6283 - loss: 0.6611
Epoch 8/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 10s 156ms/step - accuracy: 0.6335 - loss: 0.6576
Epoch 9/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 12s 176ms/step - accuracy: 0.6392 - loss: 0.6549
Epoch 10/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 12s 173ms/step - accuracy: 0.6141 - loss: 0.6686
Epoch 11/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 11s 168ms/step - accuracy: 0.6407 - loss: 0.6532
Epoch 12/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 12s 17

In [80]:
loss, accuracy = model_lstm.evaluate(np.array(validation_data), np.array(validation_labels), verbose=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6194 - loss: 0.6651


In [81]:
print(loss, accuracy)

0.6477655172348022 0.6499999761581421


## FCN - Logistic Regression Discriminator

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout

## hyperparams
init_lr = 0.1
lr_decay_rate = 0.5
lr_decay_steps = 100
dropout_p = 0.2
l2_lambda = 0.005

model_fcn = tf.keras.Sequential()
e = Embedding(len(vocab), 50, weights=[embedding_matrix], input_length = max_length, trainable=False)
model_fcn.add(e)
model_fcn.add(Flatten())
model_fcn.add(Dropout(dropout_p))
model_fcn.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(init_lr, decay_rate=lr_decay_rate, decay_steps=lr_decay_steps)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model_fcn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print(model_fcn.summary())

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)        │ ?                      │       425,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 425,250 (1.62 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 425,250 (1.62 MB)

None


In [76]:
model_fcn.fit(np.array(training_data), np.array(training_labels), epochs=75, verbose=1)

Epoch 1/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5863 - loss: 66.4531
Epoch 2/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6157 - loss: 25.3378
Epoch 3/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7675 - loss: 5.5717
Epoch 4/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7183 - loss: 4.2164
Epoch 5/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7809 - loss: 2.7233
Epoch 6/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7718 - loss: 2.3047
Epoch 7/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8035 - loss: 2.0693
Epoch 8/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8077 - loss: 1.8875
Epoch 9/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8260 - loss: 1.6823
Epoch 10/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8116 - loss: 1.7640
Epoch 11/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8155 - loss: 1.5687
Epoch 12/75
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8246 - 

In [77]:
loss, accuracy = model_fcn.evaluate(np.array(validation_data), np.array(validation_labels), verbose=1)
print(loss, accuracy)
## best so far is about 85% at around 50 loss

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8018 - loss: 1.3027
1.2511647939682007 0.8299999833106995


# Iterative Back Training

First define the two models (S --> NY) and (NY --> S)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Flatten

## hyperparams for both translation models
dropout_p = 0.2
l2_lambda = 0.005
## end hyperparams

In [10]:
model_s_to_ny = tf.keras.Sequential()
e = Embedding(len(vocab), 50, weights=[embedding_matrix], input_length = max_length, trainable=False)
model_s_to_ny.add(e)
model_s_to_ny.add(LSTM(100, input_shape = (max_length, 50)))
model_s_to_ny.add(Dropout(dropout_p))
model_s_to_ny.add(Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
model_s_to_ny.add(Dropout(dropout_p))
model_s_to_ny.add(Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
model_s_to_ny.add(Dense(len(vocab), activation='softmax'))
print(model_s_to_ny.summary())

C:\Users\Anthony\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
C:\Users\Anthony\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │       425,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 425,250 (1.62 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 425,250 (1.62 MB)

None


In [11]:
model_ny_to_s = tf.keras.Sequential()
e = Embedding(len(vocab), 50, weights=[embedding_matrix], input_length = max_length, trainable=False)
model_ny_to_s.add(e)
model_ny_to_s.add(LSTM(100, input_shape = (max_length, 50)))
model_ny_to_s.add(Dropout(dropout_p))
model_ny_to_s.add(Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
model_ny_to_s.add(Dropout(dropout_p))
model_ny_to_s.add(Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))
model_ny_to_s.add(Dense(len(vocab), activation='softmax'))

print(model_ny_to_s.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │       425,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 425,250 (1.62 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 425,250 (1.62 MB)

None


Now define the discriminator model

In [12]:

model_discriminator = tf.keras.Sequential()
e = Embedding(len(vocab), 50, weights=[embedding_matrix], input_length = max_length, trainable=False)
model_discriminator.add(e)
model_discriminator.add(Flatten())
model_discriminator.add(Dropout(dropout_p))
model_discriminator.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)))

Now, define optimizers for both the translation models and the discriminator model

In [13]:
## disc opt hyperparam
trans_init_lr = 0.1
# disc_decay_rate = 0.5
trans_decay_steps = 1000
trans_decay_alpha = 0.0

trans_lr_schedule = tf.keras.optimizers.schedules.CosineDecay(trans_init_lr, trans_decay_steps, alpha=trans_decay_alpha)
optimizer_trans = tf.keras.optimizers.Adam(learning_rate=trans_lr_schedule)

In [14]:
## disc opt hyperparam
disc_init_lr = 0.1
# disc_decay_rate = 0.5
disc_decay_steps = 1000
disc_decay_alpha = 0.0

disc_lr_schedule = tf.keras.optimizers.schedules.CosineDecay(disc_init_lr, disc_decay_steps, alpha=disc_decay_alpha)
optimizer_disc = tf.keras.optimizers.Adam(learning_rate=disc_lr_schedule)

Now, define the training loop of IBT (this is simplified from the paper, essentially just a GAN right now, it is not modeling a parallel corpus of data)

In [15]:
def train_step(x_s_to_ny, x_ny_to_s, recon_weight=1, adv_weight=1, verbose=False) :
    with tf.GradientTape(persistent = True) as tape :
        # forward pass
        # s to ny forward and reconstruction
        if verbose : print(f"Starting Shakespeare to New Yorker forward pass")
        x_s_to_ny_out = model_s_to_ny(x_s_to_ny)
        x_s_to_ny_recon = model_ny_to_s(x_s_to_ny_out)
        x_s_to_ny_out_disc = model_discriminator(x_s_to_ny_out)
        # ny to s forward and reconstruction
        if verbose : print(f"Starting New Yorker to Shakespear forward pass")
        x_ny_to_s_out = model_ny_to_s(x_ny_to_s)
        x_ny_to_s_recon = model_s_to_ny(x_ny_to_s_out)
        x_ny_to_s_out_disc = model_discriminator(x_ny_to_s_out)

        # losses
        if verbose : print(f"Starting loss calculations")
        # reconstruction loss
        loss_s_to_ny_recon = tf.keras.losses.binary_crossentropy(x_s_to_ny, x_s_to_ny_recon)
        loss_ny_to_s_recon = tf.keras.losses.binary_crossentropy(x_ny_to_s, x_ny_to_s_recon)
        loss_recon = loss_s_to_ny_recon + loss_ny_to_s_recon
        if verbose : print(f"Calculated reconstruction loss as {loss_recon}")
        # adversarial loss
        loss_s_to_ny_adv = tf.keras.losses.binary_crossentropy([0 for i in range(len(x_s_to_ny))], x_s_to_ny_out_disc)
        loss_ny_to_s_adv = tf.keras.losses.binary_crossentropy([1 for i in range(len(x_ny_to_s))], x_ny_to_s_out_disc)
        loss_adv = loss_s_to_ny_adv + loss_ny_to_s_adv
        if verbose : print(f"Calculated adversarial loss as {loss_adv}")
        # total loss for translators
        loss_trans = loss_recon*recon_weight + loss_adv*adv_weight
        if verbose : print(f"Calculated total translation loss as {loss_trans}")
        # loss for discriminators
        loss_disc = loss_adv
        if verbose : print(f"Calculated total loss for discriminator as {loss_disc}")
    
    # gradients
    if verbose : print(f"Starting to apply gradients and update parameters for the translator models")
    gradients_trans = tape.gradient(loss_trans, model_s_to_ny.trainable_variables + model_ny_to_s.trainable_variables)
    optimizer_trans.apply_gradients(zip(gradients_trans, model_s_to_ny.trainable_variables + model_ny_to_s.trainable_variables))
    if verbose : print(f"Finished updating the translator models")

    if verbose : print(f"Starting to apply gradients and update parameters for the discriminator model")
    gradients_disc = tape.gradient(loss_disc, model_discriminator.trainable_variables)
    optimizer_disc.apply_gradients(zip(gradients_disc, model_discriminator.trainable_variables))
    if verbose : print(f"Finished updating the discriminator model")

    if verbose : print(f"Finishing training step.\n")
    return loss_trans, loss_disc



Now, train over some batches for a few epochs

In [20]:
dataset = tf.data.Dataset.range(100).batch(32)

# Get the number of batches
num_batches = tf.data.experimental.cardinality(dataset).numpy()
print("Number of batches:", num_batches)

Number of batches: 4


In [23]:
raw_x_s_to_ny = [training_data[i] for i in range(len(training_labels)) if labels[i] == 1]
raw_x_ny_to_s = [training_data[i] for i in range(len(training_labels)) if labels[i] == 0]

print(f"x_s_to_ny shape is {np.array(raw_x_s_to_ny).shape}")
print(f"x_ny_to_s shape is {np.array(raw_x_ny_to_s).shape}")

ds_s_to_ny = tf.data.Dataset.from_tensor_slices(
    raw_x_s_to_ny
)
ds_ny_to_s = tf.data.Dataset.from_tensor_slices(
    raw_x_ny_to_s
)

ds_s_to_ny = ds_s_to_ny.batch(np.array(raw_x_s_to_ny).shape[0] // 20)
ds_ny_to_s = ds_ny_to_s.batch(np.array(raw_x_ny_to_s).shape[0] // 20)

for batch in ds_s_to_ny.take(1):  # Take the first batch
    print("Batch size for s to ny:", batch.shape[0])
num_batches = tf.data.experimental.cardinality(ds_s_to_ny).numpy()
print("Number of batches:", num_batches)

for batch in ds_ny_to_s.take(1):  # Take the first batch
    print("Batch size for ny to s:", batch.shape[0])
num_batches = tf.data.experimental.cardinality(ds_ny_to_s).numpy()
print("Number of batches:", num_batches)

combined_ds = tf.data.Dataset.zip((ds_s_to_ny, ds_ny_to_s))
print(combined_ds)

x_s_to_ny shape is (711, 50)
x_ny_to_s shape is (1412, 50)
Batch size for s to ny: 35
Number of batches: 21
Batch size for ny to s: 70
Number of batches: 21
<_ZipDataset element_spec=(TensorSpec(shape=(None, 50), dtype=tf.int32, name=None), TensorSpec(shape=(None, 50), dtype=tf.int32, name=None))>


In [24]:
for epoch in range(10) : # num epochs
    print(f"Epoch {epoch + 1} starting...")
    curr_loss_trans = 0
    curr_loss_disc = 0
    for x_s_to_ny, x_ny_to_s in combined_ds :
        loss_trans, loss_disc = train_step(training_data, x_ny_to_s, recon_weight=1, adv_weight=1, verbose=True)
        curr_loss_trans += loss_trans
        curr_loss_disc += loss_disc
        print(f"   Batch Translator Loss: {loss_trans}\n   Batch Discriminator Loss: {loss_disc}")
    print(f"Total Epoch Translator Loss: {curr_loss_trans}\nTotal Epoch Discriminator Loss: {curr_loss_disc}\n")

Epoch 1 starting...
Starting Shakespeare to New Yorker forward pass


: 